# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')


In [2]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# # Table and definitions sample
# data = {'table': # ENTER A TABLE COLUMNS HERE,
#         'definition': # ENTER A TABLE DEFINITATIONS HERE
# df = pd.DataFrame(data)
# print(df)

data = {
    'table': ['products', 'customers', 'sales'],
    'definition': [
        'product_id, name, price, quantity',
        'customer_id, name, address',
        'sale_id, product_id, customer_id, quantity, sale_date'
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0   products                  product_id, name, price, quantity
1  customers                         customer_id, name, address
2      sales  sale_id, product_id, customer_id, quantity, sa...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

- `df.iterrows()` → loops through each row of your DataFrame.

- `df.iterrows()` returns two things per loop:

    1. `index` → the row number (0, 1, 2, …)

    2. `row` → a Series object that contains the actual row data.
- `\n'.join([f"{row['table']}: {row['definition']}" ` → joins everything in a big string

In [6]:
print(text_tables)

products: product_id, name, price, quantity
customers: customer_id, name, address
sales: sale_id, product_id, customer_id, quantity, sale_date


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [9]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='How many products did the custome 1 bought?')

In [10]:
print(return_OAI(pqt1)) # sends that full prompt to OpenAI’s GPT model

{
    "tables": ["products", "sales"]
}


In [14]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="write a query which will find the 3 best customers based on the number of the products they bought and based on my tables.")

In [15]:
print(return_OAI(pqt3))

{
  "tables": ["customers", "sales"]
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [16]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What are the top 3 sales?")
print(return_OAI(pqt3))

```json
{
    "tables": ["sales", "products", "customers"]
}
```


Wrong!

In [17]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What are the top 3 buyers based on the sales")
print(return_OAI(pqt3))

{
  "tables": ["customers", "sales"]
}


In [18]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What are the top 3 buyers based on the number of products they bought")
print(return_OAI(pqt3))

{
    "tables": ["customers", "sales"]
}
